# Imports

In [61]:
import pandas as pd
import numpy as np
import re
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from itertools import product

# Input de DataSet

In [62]:
df = pd.read_csv('urbansounds_features.csv')
df

,chroma_stft,chroma_cqt,chroma_cens,melspectogram,rms,centroid,bandwidth,contrast,flatness,rolloff,...,mcffs_32,mcffs_33,mcffs_34,mcffs_35,mcffs_36,mcffs_37,mcffs_38,mcffs_39,mcffs_40,Label
0,0.494262,0.568759,0.267266,3.465249,0.068657,2177.483658,3649.518344,19.685466,0.002088,4745.917969,...,-2.700133,-0.350788,-0.123943,-0.308717,0.377525,-0.907509,-0.254973,-1.738416,-1.207377,101415-3-0-2.wav
1,0.422076,0.556903,0.270826,3.132792,0.076706,1623.980658,2897.277104,21.508354,0.001123,2743.891814,...,-5.048006,-1.626652,-1.303208,-1.028367,0.470883,-2.959822,-1.439514,-2.282194,-1.155120,101415-3-0-3.wav
2,0.575613,0.606710,0.275953,2.257988,0.044251,2615.200756,4200.855657,18.001559,0.003370,6143.329229,...,-1.015445,0.989107,1.142546,0.407650,-0.166601,-0.859933,-0.637618,-1.657652,-1.639226,101415-3-0-8.wav
3,0.175040,0.526637,0.258603,1.481219,0.043195,1223.789196,1367.391253,16.179755,0.583564,2473.821756,...,2.447487,-1.956081,0.440909,-1.384400,0.585570,-0.601787,-0.864582,-0.896903,-0.161344,102106-3-0-0.wav
4,0.412279,0.608818,0.264944,0.806509,0.035636,1470.863386,2296.662162,12.349992,0.353831,3264.932914,...,0.232175,0.326373,0.610677,0.570734,1.030308,0.507107,0.160745,0.642600,0.364088,102305-6-0-0.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8727,0.479371,0.552483,0.266297,2.410669,0.127638,2431.943128,2504.586835,16.611774,0.000412,4677.947945,...,0.199730,-0.839973,-4.665137,-0.551395,0.214190,0.865874,1.078110,-3.648149,0.705190,99500-2-0-23.wav
8728,0.545358,0.593931,0.273223,2.837043,0.130824,1979.367181,2215.734460,16.131903,0.000247,3817.306386,...,0.332412,2.602965,-0.238956,2.640356,-0.004591,-0.695137,-0.639137,-2.883146,0.214829,99500-2-0-29.wav
8729,0.436206,0.605989,0.275923,5.695730,0.190655,2192.398026,2140.849300,17.080651,0.000198,4041.189283,...,0.285693,1.105075,-2.979676,0.069636,0.038741,0.715938,2.593126,-2.714100,1.224722,99500-2-0-39.wav
8730,0.524807,0.671077,0.277764,3.745609,0.153592,2187.172172,2190.171430,16.642445,0.000244,4042.500000,...,-0.099813,0.535876,-0.444768,2.225560,1.202731,-1.514751,1.340369,-0.692652,1.283734,99500-2-0-41.wav


Clean the DataSet

In [63]:
def calculate_mean_from_string(string):
    cleaned_string = string.replace('\n', '')
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", cleaned_string)
    array = np.array(numbers, dtype=float)
    mean_value = np.mean(array)
    return mean_value

In [64]:
for column in df.columns:
    if column != 'Label':
        if df[column].dtype != float and df[column].dtype != int:
            df[column] = df[column].apply(calculate_mean_from_string)
    else:
        df[column] = df[column].str.split('-').str[1].astype(int)
df

,chroma_stft,chroma_cqt,chroma_cens,melspectogram,rms,centroid,bandwidth,contrast,flatness,rolloff,...,mcffs_32,mcffs_33,mcffs_34,mcffs_35,mcffs_36,mcffs_37,mcffs_38,mcffs_39,mcffs_40,Label
0,0.494262,0.568759,0.267266,3.465249,0.068657,2177.483658,3649.518344,19.685466,0.002088,4745.917969,...,-2.700133,-0.350788,-0.123943,-0.308717,0.377525,-0.907509,-0.254973,-1.738416,-1.207377,3
1,0.422076,0.556903,0.270826,3.132792,0.076706,1623.980658,2897.277104,21.508354,0.001123,2743.891814,...,-5.048006,-1.626652,-1.303208,-1.028367,0.470883,-2.959822,-1.439514,-2.282194,-1.155120,3
2,0.575613,0.606710,0.275953,2.257988,0.044251,2615.200756,4200.855657,18.001559,0.003370,6143.329229,...,-1.015445,0.989107,1.142546,0.407650,-0.166601,-0.859933,-0.637618,-1.657652,-1.639226,3
3,0.175040,0.526637,0.258603,1.481219,0.043195,1223.789196,1367.391253,16.179755,0.583564,2473.821756,...,2.447487,-1.956081,0.440909,-1.384400,0.585570,-0.601787,-0.864582,-0.896903,-0.161344,3
4,0.412279,0.608818,0.264944,0.806509,0.035636,1470.863386,2296.662162,12.349992,0.353831,3264.932914,...,0.232175,0.326373,0.610677,0.570734,1.030308,0.507107,0.160745,0.642600,0.364088,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8727,0.479371,0.552483,0.266297,2.410669,0.127638,2431.943128,2504.586835,16.611774,0.000412,4677.947945,...,0.199730,-0.839973,-4.665137,-0.551395,0.214190,0.865874,1.078110,-3.648149,0.705190,2
8728,0.545358,0.593931,0.273223,2.837043,0.130824,1979.367181,2215.734460,16.131903,0.000247,3817.306386,...,0.332412,2.602965,-0.238956,2.640356,-0.004591,-0.695137,-0.639137,-2.883146,0.214829,2
8729,0.436206,0.605989,0.275923,5.695730,0.190655,2192.398026,2140.849300,17.080651,0.000198,4041.189283,...,0.285693,1.105075,-2.979676,0.069636,0.038741,0.715938,2.593126,-2.714100,1.224722,2
8730,0.524807,0.671077,0.277764,3.745609,0.153592,2187.172172,2190.171430,16.642445,0.000244,4042.500000,...,-0.099813,0.535876,-0.444768,2.225560,1.202731,-1.514751,1.340369,-0.692652,1.283734,2


# Classification

In [25]:
X = df.drop('Label', axis=1) 
y = df['Label']

In [26]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
# Print class distribution before SMOTE
print("Class distribution before SMOTE:")
print(pd.Series(y_train).value_counts())

Class distribution before SMOTE:
Label
9    811
5    807
2    803
4    798
7    791
0    789
3    783
8    749
1    352
6    302
Name: count, dtype: int64


In [28]:
# Oversample the features values using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [29]:
# Print class distribution after SMOTE
print("\nClass distribution after SMOTE:")
print(pd.Series(y_train_resampled).value_counts())


Class distribution after SMOTE:
Label
2    811
9    811
5    811
1    811
7    811
0    811
8    811
4    811
6    811
3    811
Name: count, dtype: int64


In [30]:
# Standardize the feature values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# Using TenserFlow

In [65]:
mean_neurons = (X_train_scaled.shape[1] + len(np.unique(y_train_resampled))) // 2
num_input_neurons = X_train_scaled.shape[1]
num_output_neurons = len(np.unique(y_train_resampled))
neurons_hidden_layer = int(2/3 * num_input_neurons + 1/3 * num_output_neurons)

38


In [32]:
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

cv_scores = []

for train_index, val_index in stratified_kfold.split(X_train_scaled, y_train_resampled):
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train_resampled[train_index], y_train_resampled[val_index]

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=mean_neurons, input_shape=(num_input_neurons,)),
        tf.keras.layers.Dense(units=neurons_hidden_layer),
        tf.keras.layers.Dense(units=num_output_neurons, activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy')

    model.fit(X_train_fold, y_train_fold, epochs=300, validation_data=(X_val_fold, y_val_fold))

    y_val_pred_probs = model.predict(X_val_fold)
    y_val_pred = np.argmax(y_val_pred_probs, axis=1)

    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)
    cv_scores.append(fold_accuracy)

average_cv_accuracy = np.mean(cv_scores)
print(f'Average Cross-Validation Accuracy: {average_cv_accuracy}')

Epoch 1/300
229/229 [==============================] - 0s 685us/step - loss: 1.4701 - val_loss: 1.0554
Epoch 2/300
229/229 [==============================] - 0s 425us/step - loss: 1.0022 - val_loss: 0.8859
Epoch 3/300
229/229 [==============================] - 0s 451us/step - loss: 0.8873 - val_loss: 0.8392
Epoch 4/300
229/229 [==============================] - 0s 446us/step - loss: 0.8346 - val_loss: 0.8103
Epoch 5/300
229/229 [==============================] - 0s 420us/step - loss: 0.8067 - val_loss: 0.7840
Epoch 6/300
229/229 [==============================] - 0s 439us/step - loss: 0.7882 - val_loss: 0.7752
Epoch 7/300
229/229 [==============================] - 0s 433us/step - loss: 0.7733 - val_loss: 0.7852
Epoch 8/300
229/229 [==============================] - 0s 415us/step - loss: 0.7618 - val_loss: 0.7759
Epoch 9/300
229/229 [==============================] - 0s 418us/step - loss: 0.7539 - val_loss: 0.7458
Epoch 10/300
229/229 [==============================] - 0s 415us/step - l

Now let's try to find the best dropout and activation functions for the model

```
def grid_search_best_parameters():
    best_avg_accuracy = 0
    best_parameters = {}

    dropout_values = np.arange(0, 1, 0.1)
    activation_functions = ['relu', 'sigmoid', 'exponential']

    total_runs = len(dropout_values) * len(activation_functions) * len(activation_functions)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)  # Adjust the number of folds as needed

    with tqdm(total=total_runs, desc="Grid Search Progress", position=0, leave=True) as pbar:
        for i, dropout in enumerate(dropout_values):
            for function1 in activation_functions:
                for function2 in activation_functions:
                    avg_accuracy = 0

                    for train_index, test_index in skf.split(X_train_scaled, y_train_resampled):
                        X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
                        y_train_cv, y_test_cv = y_train_resampled[train_index], y_train_resampled[test_index]

                        # Build an MLP with dropout using TensorFlow 
                        m = tf.keras.Sequential([
                            tf.keras.layers.Dense(units=mean_neurons, activation=function1, input_shape=(X_train_scaled.shape[1],)),
                            tf.keras.layers.Dropout(dropout),
                            tf.keras.layers.Dense(units=neurons_hidden_layer, activation=function2),
                            tf.keras.layers.Dropout(dropout),
                            tf.keras.layers.Dense(units=len(np.unique(y_train_resampled)), activation='softmax')
                        ])

                        # Compile the model
                        m.compile(loss='sparse_categorical_crossentropy')

                        # Train the model
                        m.fit(X_train_cv, y_train_cv, epochs=300, validation_data=(X_test_scaled, y_test), verbose=0)

                        # Evaluate the model on the test set
                        y_pred_prob = m.predict(X_test_cv)
                        y_pred = np.argmax(y_pred_prob, axis=1)

                        # Accuracy calculation
                        accuracy = accuracy_score(y_test_cv, y_pred)
                        avg_accuracy += accuracy

                    avg_accuracy /= skf.get_n_splits()

                    # Check if the current parameters result in a higher average accuracy
                    if avg_accuracy > best_avg_accuracy:
                        best_avg_accuracy = avg_accuracy
                        best_parameters = {
                            'best_dropout': dropout,
                            'best_function1': function1,
                            'best_function2': function2,
                            'best_avg_accuracy': best_avg_accuracy
                        }

                    # Update progress bar
                    pbar.update(1)

    return best_parameters

```

```
parameters = grid_search_best_parameters()
print(parameters)
```

![](act_funct.png)

In [ ]:
dropout = 0.1 # parameters['best_dropout']
function1 = 'relu' # parameters['best_function1']
function2 = 'relu' # parameters['best_function2']

In [ ]:
function = 'relu'
optimizer = 'adam'
loss_function = 'sparse_categorical_crossentropy'
batch_size = 32
num_epochs = 300
learning_rate = 0.001

Now lets train the model with the best parameters

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

cv_scores = []

for train_index, val_index in stratified_kfold.split(X_train_scaled, y_train_resampled):
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train_resampled[train_index], y_train_resampled[val_index]

    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=mean_neurons, activation=function, input_shape=(num_input_neurons,)),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(units=neurons_hidden_layer, activation=function),
        tf.keras.layers.Dropout(dropout),
        tf.keras.layers.Dense(units=num_output_neurons, activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy')

    model.fit(X_train_fold, y_train_fold, epochs=300, validation_data=(X_val_fold, y_val_fold))

    y_val_pred_probs = model.predict(X_val_fold)
    y_val_pred = np.argmax(y_val_pred_probs, axis=1)

    fold_accuracy = accuracy_score(y_val_fold, y_val_pred)
    cv_scores.append(fold_accuracy)

average_cv_accuracy = np.mean(cv_scores)
print(f'Average Cross-Validation Accuracy: {average_cv_accuracy}')

Now lets change the training strategies

In [ ]:
def grid_search_best_parameters():
    best_avg_accuracy = 0
    best_parameters = {}

    optimizer = ['adam', 'sgd', 'adagrad']
    learning_rate = ['0.005','0.01' '0.1']
    batch_size = [32, 64]

    total_runs = len(optimizer) * len(learning_rate) * len(batch_size)

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    with tqdm(total=total_runs, desc="Grid Search Progress", position=0, leave=True) as pbar:
        for params in product(optimizer_values, learning_rate_values, batch_size_values):
            optimizer, learning_rate, batch_size = params
            avg_accuracy = 0

            for train_index, test_index in skf.split(X_train_scaled, y_train_resampled):
                X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
                y_train_cv, y_test_cv = y_train_resampled[train_index], y_train_resampled[test_index]

                model = tf.keras.Sequential([
                    tf.keras.layers.Dense(units=mean_neurons, activation='relu', input_shape=(X_train_scaled.shape[1],)),
                    tf.keras.layers.Dropout(dropout),
                    tf.keras.layers.Dense(units=neurons_hidden_layer, activation='relu'),
                    tf.keras.layers.Dropout(dropout),
                    tf.keras.layers.Dense(units=len(np.unique(y_train_resampled)), activation='softmax')
                ])

                # Compile the model with the specified optimizer and learning rate
                model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, learning_rate=learning_rate)

                # Train the model
                model.fit(X_train_cv, y_train_cv, epochs=300, validation_data=(X_test_scaled, y_test), batch_size=batch_size, verbose=0)

                # Evaluate the model on the test set
                y_pred_prob = model.predict(X_test_cv)
                y_pred = np.argmax(y_pred_prob, axis=1)

                # Accuracy calculation
                accuracy = accuracy_score(y_test_cv, y_pred)
                avg_accuracy += accuracy

            avg_accuracy /= skf.get_n_splits()

            # Check if the current parameters result in a higher average accuracy
            if avg_accuracy > best_avg_accuracy:
                best_avg_accuracy = avg_accuracy
                best_parameters = {
                    'best_optimizer': optimizer,
                    'best_learning_rate': learning_rate,
                    'best_batch_size': batch_size,
                    'best_avg_accuracy': best_avg_accuracy
                }

            # Update progress bar
            pbar.update(1)

    return best_parameters


In [ ]:
early_stopping = {'True', 'False'}
weight_regularization = {'True', 'False'}
data_augmentation = {'True', 'False'}

# Using Scikit-Learn

In [ ]:
# Create an MLP classifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(mean_neurons,), activation="relu", max_iter=300)
mlp_classifier.fit(X_train_scaled, y_train_resampled)
y_pred = mlp_classifier.predict(X_test_scaled)

```
### CNN
# Reshape data for CNN
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1], 1))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1], 1))

# Convert labels to categorical one-hot encoding
y_train_onehot = to_categorical(y_train_encoded)
y_test_onehot = to_categorical(y_test_encoded)
# Define the CNN model with different activation functions for hidden layers
activation_functions = ['tanh', 'relu', 'sigmoid']

for activation1 in activation_functions:
    for activation2 in activation_functions:
        # Define the CNN model
        model = Sequential()
        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(1, X_train_scaled.shape[1], 1)))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation=activation1))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation=activation2))
        model.add(Flatten())
        model.add(Dense(64, activation=activation1))
        model.add(Dense(y_train_onehot.shape[1], activation='sigmoid'))  # Sigmoid for the output layer
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
model.fit(X_train_reshaped, y_train_onehot, epochs=10, validation_split=0.2)
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test_reshaped, y_test_onehot)
print(f'Test 
```